Here is the solution for the triangle integration code which simply breaks up the loop into multiple blocks.  The plan is simply to split the generator `comb` into multiple blocks.  First we count the number of iterations by simploy runnig through `comb` once and counting them.  Then we use `divide_loops` to split it up and then `gen` to return a new generator just for the part we need.

In [ ]:
"""
    This code computes the value of the integral:
    $E_{i,j,k} = \sum_{l_1}\sum_{l_2}\sum_{l_3}
    \left( \int_{-1}^{1} d\mu P_{l_1}(\mu) P_{l_2}(\mu)
    P_{l_3}(\mu)\right) X^i_{l_1} X^j_{l_2} X^k_{l_3} $
    
    The code takes some set of $X^i_{l}$ for i element of [0,imax]
    and calculates $E_{i,j,k}$ for every possible triple of [0,imax]
    
    The code works by reordering the integration and sum to form:
    $Y^i(\mu) = \sum_l P_{l}(\mu)  X^i_{l}$
    $E_{i,j,k} = \int_{-1}^{1} d\mu Y^i(\mu)Y^j(\mu)Y^k(\mu)$
    
    The first is computed as a matix product
    The second is computed using Gauss Legendre quadriture
    as it is exact for polynomial arguments
    
    Written by James Fergusson: J.Fergusson@DAMTP.cam.ac.uk
    """

# Load modules:
import numpy as np
from itertools import combinations_with_replacement
from mpi4py import MPI

def legendre_array(X,lmax):
    """
        Calculates an array of Legendre polynomials
        with size: len(X) by lmax
        
        Parameters
        ----------
        lmax: int
        the maximum degree, must be positive
        X: array(float)
        points to calculate the polynomials on
        
        Returns
        ----------
        array(float)
        A table of the Legendre polynomials
        """
    
    # Initilise the array
    P = np.zeros((lmax,len(X)), dtype='float')
    
    # Set P_0 and P_l
    P[0,:]=1.0
    P[1,:]=X
    
    # Calculate the rest via iteration
    for n in range(2,lmax):
        P[n,:] = ((2*n-1)*X*P[n-1,:] - (n-1)*P[n-2,:])/n
    
    # return the array of Legendre polynomials
    return P

def divide_loops(loops,size):
    """
    Divides 'loops' into 'size' groups
    
    Divide the number of loops into as close to equal
    size chunks as possible for use with parallelisation
    
    
    Parameters
    ----------
    loops: int
        the number of loops to divide up
    size: int
        the number of groups to break loops into
        
    Returns
    ----------
    start_loop: int
        The loop number to start from
    end_loop: int
        The loop number to stop at 
    """
    # floor division
    loop_rank=loops//size
    # remainder
    auxloop = loops%size
    #calculate start and end
    start_loop = rank*loop_rank
    end_loop = (rank+1)*loop_rank
    
    # allocate remainder across loops
    if(auxloop!=0):
        if (rank < auxloop):
            start_loop = start_loop + rank
            end_loop = end_loop + rank + 1
        else:
            start_loop = start_loop + auxloop
            end_loop = end_loop + auxloop
    # return start and end
    return start_loop, end_loop

def gen(s,e,comb):
    """
    Create generator for part of triangular triple sum
    
    Parameters
    ----------
    s: int
        the loop number to begin at
    e: int
        the loop number to end at
        
    Returns
    ----------
    None: 
    
    """
    # initilise count
    count = 1
    #triangular loop
    for i,j,k in comb:
        if count>e:
            break
        # Don't return triple until we have gone s loops
        if count>s:
            yield i,j,k
        count += 1

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print( "[{}] Starting".format(rank) )

# Set maximums for l and i.
lmax = 50
imax = 5

# Compute number of points requiredfor the Gauss-Legendre
# quadtiture to be exact for a given lmax.
npts = 3*lmax//2+1

# Calculate Gauss-Legendre quadriture points, X, and weights, W.
X, W = np.polynomial.legendre.leggauss(npts)

# Create arrays for Legenre polynomials
P = legendre_array(X,lmax)


# Create Chebyshev Polynomials
# This could be changed to accept a general function or read from disk
# in future versions
L = np.linspace(-1,1,lmax)
C = legendre_array(L,imax)

# Compute the sum over L using a matrix product
Y = np.dot(C,P)

# Construct an iterator over possible triples of i
comb = combinations_with_replacement(range(imax),3)

# Count number of iterations the lazy way
n=0
for i,j,k in comb:
    n +=1

# divide the loops up
comb = combinations_with_replacement(range(imax),3)
s,e = divide_loops(n,size)
# Select sub section
comb_mpi = gen(s,e,comb)


# Calculate the integral (just a sum here) for each triple and print
# the result

# *******************************************************************
# *******************************************************************
# *****         Edit below here to parallelise the for          *****
# *******************************************************************
# *******************************************************************

for i,j,k in comb_mpi:
    z = np.sum(W[:]*Y[i,:]*Y[j,:]*Y[k,:])
    print("({},{},{}) produces {:5g}".format(i,j,k,z))


Here is the solution for the Game of Life code over 4 processors.  Here we divide the 10x40 grid into 4 10x10 blocks.  In order for the convolution to work correctly we need to know what the neighbours are for the cells on the left and right edges. To do this we pad each block to be 10x12 then communicate the left and right edge cells to the ranks before and after.  Now we can perform the convolution.

In [ ]:
from mpi4py import MPI
import numpy as np
import scipy.signal as scs
import matplotlib.pyplot as plt
import sys

# Function that advances the game one step
def step(cells):
    mask = np.array([[1,1,1],[1,0,1],[1,1,1]])
    count = scs.convolve2d(cells,mask,mode='same',boundary='wrap')
    newcells = np.where(cells==1,np.where((count>1),np.where(count<4,1,0),0),np.where(count==3,1,0))
    return newcells

# Set up communication
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# 12 is so there is space for the cells from other ranks
cells = np.zeros((10,12))
if rank==0:
    cells[3,6:8] = 1
    cells[4,4:6] = 1
    cells[4,7:9] = 1
    cells[5,4:8] = 1
    cells[6,5:7] = 1

# Find name of neighbouring ranks
rank_fwd = (rank+1)%size
rank_bck = (rank-1)%size

# Create blank data array to recieve to
data_rcv = np.zeros((10), dtype='int')

for i in range(60):
    
    # Send left part of the cell backards to make the right side of the previous array
    data_snd = np.copy(cells[:,1])
    comm.Sendrecv(data_snd, dest=rank_bck, recvbuf=data_rcv, source=rank_fwd)
    cells[:,11] = data_rcv
    
    # Send right part of the cell forwards to make the left side of the next array
    data_snd = np.copy(cells[:,10])
    comm.Sendrecv(data_snd, dest=rank_fwd, recvbuf=data_rcv, source=rank_bck)
    cells[:,0] = data_rcv
    
    # Step the data
    cells = step(cells)

# Plot results, duck should be in 4th rank (number 3)
fig = plt.figure()
ax = plt.axes()
ax.matshow(cells[:,1:11])
filename = 'PlotRank'+str(rank)+'.png'
plt.savefig(filename)

This is a 1d communication topology and it's fairly simple.  We could easily imagine that if we instead had a 20x20 grid we may have decided to split it into 4 10x10 blocks but now the communication would be much more complicated as we would need to get edge cells from the: top, bottom, left, right, top-left, top-right, bottom-left and bottom-right ranks.  This is where topologies can come in useful but there is no `shift` function for diagonals so we will skip it here.  Here is an example of what the communication would look like:

In [ ]:
from mpi4py import MPI
import numpy as np
import scipy.signal as scs
import matplotlib.pyplot as plt
import sys

# Function that advances the game one step
def step(cells):
    mask = np.array([[1,1,1],[1,0,1],[1,1,1]])
    count = scs.convolve2d(cells,mask,mode='same',boundary='wrap')
    newcells = np.where(cells==1,np.where((count>1),np.where(count<4,1,0),0),np.where(count==3,1,0))
    return newcells

# Set up communication
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# 12 is so there is space for the cells from other ranks
cells = np.zeros((12,12))
if rank==0:
    cells[4,6:9] = 1
    cells[9,6:9] = 1
    cells[6:9,4] = 1
    cells[6:9,9] = 1
if rank==1:
    cells[4,2:5] = 1
    cells[9,2:5] = 1
    cells[6:9,1] = 1
    cells[6:9,6] = 1
if rank==2:
    cells[1,6:9] = 1
    cells[6,6:9] = 1
    cells[2:5,4] = 1
    cells[2:5,9] = 1
if rank==3:
    cells[1,2:5] = 1
    cells[6,2:5] = 1
    cells[2:5,1] = 1
    cells[2:5,6] = 1

# Find name of neighbouring ranks
# Topology is:
#     0 1
#     2 3
sizex = 2
sizey = 2

def find_rank(rank,sizex,sizey,vec):
    clm = rank%sizey
    row = rank//sizex
    new_row = (row+vec[0])%sizex
    new_clm = (clm+vec[1])%sizey
    dest_rank = sizex*new_row+new_clm
    return dest_rank
    

rank_lf = find_rank(rank,sizex,sizey,[0,-1]) #left
rank_rt = find_rank(rank,sizex,sizey,[0,1]) #right
rank_up = find_rank(rank,sizex,sizey,[-1,0]) #up
rank_dn = find_rank(rank,sizex,sizey,[1,0]) #down
rank_ul = find_rank(rank,sizex,sizey,[-1,-1]) #up-left
rank_ur = find_rank(rank,sizex,sizey,[-1,1]) #up-right
rank_bl = find_rank(rank,sizex,sizey,[1,-1]) #down-left
rank_br = find_rank(rank,sizex,sizey,[1,1]) #down-right

# Create blank data array to recieve to for sides and corners
data_side_rcv = np.zeros((12), dtype='int')
data_corn_rcv = np.zeros((1), dtype='int')

for i in range(3):
    
    # Send left
    data_snd = np.copy(cells[:,1])
    comm.Sendrecv(data_snd, dest=rank_lf, recvbuf=data_side_rcv, source=rank_rt)
    cells[:,11] = data_side_rcv
    
    # Send right
    data_snd = np.copy(cells[:,10])
    comm.Sendrecv(data_snd, dest=rank_rt, recvbuf=data_side_rcv, source=rank_lf)
    cells[:,0] = data_side_rcv
    
    # Send top
    data_snd = np.copy(cells[1,:])
    comm.Sendrecv(data_snd, dest=rank_up, recvbuf=data_side_rcv, source=rank_dn)
    cells[11,:] = data_side_rcv
    
    # Send bottom
    data_snd = np.copy(cells[10,:])
    comm.Sendrecv(data_snd, dest=rank_dn, recvbuf=data_side_rcv, source=rank_up)
    cells[0,:] = data_side_rcv
    
    # send TL corner
    data_snd = np.copy(cells[1,1])
    comm.Sendrecv(data_snd, dest=rank_ul, recvbuf=data_corn_rcv, source=rank_br)
    cells[11,11] = data_corn_rcv
    
    # send TR corner
    data_snd = np.copy(cells[1,10])
    comm.Sendrecv(data_snd, dest=rank_ur, recvbuf=data_corn_rcv, source=rank_bl)
    cells[11,0] = data_corn_rcv
    
    # send BL corner
    data_snd = np.copy(cells[10,1])
    comm.Sendrecv(data_snd, dest=rank_bl, recvbuf=data_corn_rcv, source=rank_ur)
    cells[0,11] = data_corn_rcv
    
    # send BR corner
    data_snd = np.copy(cells[10,10])
    comm.Sendrecv(data_snd, dest=rank_br, recvbuf=data_corn_rcv, source=rank_ul)
    cells[0,0] = data_corn_rcv
    
    # Step the data
    cells = step(cells)

# Plot results
fig = plt.figure()
ax = plt.axes()
ax.matshow(cells[1:11,1:11])
filename = 'PlotRank'+str(rank)+'.png'
plt.savefig(filename)

It is easy to see how much more complicated this would get in 3D.  This is why often in higher dimensional cases we still stick with 1D communication (ie 4 blocks of 5x20 rather than 10x10) as it's both simpler and also we can easily split the grid up over any number of ranks which for higher dimenions is tricky.